In [428]:
import pandas as pd
import numpy as np

In [429]:
pd.options.display.max_columns= 1000

In [430]:
# process dp05 data

In [431]:
acs_dp05 = pd.read_csv('E:/CR2/Repos/TNC-Demand-Model-Southeast/Raw_Inputs/ACS/DP05/DP05.csv')

In [432]:
acs_dp05[['TRACT', 'COUNTY', 'STATE']] = acs_dp05.NAME.str.split(',', expand=True)

In [433]:
acs_dp05 = acs_dp05[acs_dp05['COUNTY'] == ' Fayette County']

In [434]:
acs_dp05['CENSUS_TRACT'] = acs_dp05.apply(lambda row: row['GEO_ID'][9:], axis = 1)

In [435]:
acs_dp05 = acs_dp05.rename(columns={'DP05_0018E': 'MEDIAN_AGE'})

In [436]:
acs_dp05 = acs_dp05[['MEDIAN_AGE', 'CENSUS_TRACT']]

In [437]:
# Process S1501 Data

In [438]:
acs_s1501 = pd.read_csv('E:/CR2/Repos/TNC-Demand-Model-Southeast/Raw_Inputs/ACS/S1501/S1501.csv')

In [439]:
acs_s1501[['TRACT', 'COUNTY', 'STATE']] = acs_s1501.NAME.str.split(',', expand=True)

In [440]:
acs_s1501 = acs_s1501[acs_s1501['COUNTY'] == ' Fayette County']

In [441]:
acs_s1501['CENSUS_TRACT'] = acs_s1501.apply(lambda row: row['GEO_ID'][9:], axis = 1)

In [442]:
keep = ['S1501_C01_001E', 'S1501_C01_002E', 'S1501_C01_003E', 'S1501_C01_004E', 'S1501_C01_005E', 'S1501_C01_006E', 'S1501_C01_007E', 'S1501_C01_008E', 'S1501_C01_009E', 'S1501_C01_010E', 'S1501_C01_011E', 'S1501_C01_012E', 'S1501_C01_013E', 'S1501_C01_014E', 'S1501_C01_015E','CENSUS_TRACT']

In [443]:
rename = ['TOTAL_POP_18_24', 'TOTAL_POP_18_24_NO_HIGH_SCHOOL', 'TOTAL_POP_18_24_HIGH_SCHOOL', 'TOTAL_POP_18_24_SOME_COLLEGE', 'TOTAL_POP_18_24_SOME_BACHELORS_HIGHER', 'TOTAL_POP_25P', 'TOTAL_POP_25P_LESS_THAN_9TH', 'TOTAL_POP_25P_9TH_TO_12TH', 'TOTAL_POP_25P_HIGH_SCHOOL', 'TOTAL_POP_25P_SOME_COLLEGE', 'TOTAL_POP_25P_ASSOCIATES', 'TOTAL_POP_25P_BACHELORS', 'TOTAL_POP_25P_GRADUATE', 'TOTAL_POP_25P_HIGH_SCHOOL_HIGHER', 'TOTAL_POP_25P_BACHELORS_HIGHER','CENSUS_TRACT']

In [444]:
acs_s1501 = acs_s1501[keep]
acs_s1501.columns = rename

In [445]:
# combine bachelors+ columns and drop the rest

In [446]:
acs_s1501['TOTAL_POP_25P_GRADUATE'] = acs_s1501['TOTAL_POP_25P_GRADUATE'].astype(float)
acs_s1501['TOTAL_POP_25P_BACHELORS_HIGHER'] = acs_s1501['TOTAL_POP_25P_BACHELORS_HIGHER'].astype(float)

In [447]:
acs_s1501['P_BACH_25P'] = (acs_s1501['TOTAL_POP_25P_GRADUATE'] / acs_s1501['TOTAL_POP_25P_BACHELORS_HIGHER']) * 100

In [448]:
acs_s1501 = acs_s1501[['CENSUS_TRACT', 'P_BACH_25P']]

In [449]:
# processes vehicles X income per household file 

In [450]:
veh_hhld = pd.read_csv('E:/CR2/Repos/TNC-Demand-Model-Southeast/Raw_Inputs/ACS/KY_A112306.csv', skiprows=6)

In [451]:
veh_hhld[['TRACT', 'COUNTY', 'STATE']] = veh_hhld.Output.str.split(',', expand=True)

In [452]:
veh_hhld = veh_hhld[veh_hhld['COUNTY'] == ' Fayette County']

In [453]:
low_inc_0 = ['Estimate.10', 'Estimate.11', 'Estimate.12', 'Estimate.13', 'Estimate.14', 'Estimate.15']

In [454]:
high_inc_0 = ['Estimate.16', 'Estimate.17']

In [455]:
low_inc_1p = ['Estimate.19', 'Estimate.20', 'Estimate.21', 'Estimate.22', 'Estimate.23', 'Estimate.24']

In [456]:
high_inc_1p = ['Estimate.30', 'Estimate.31']

In [457]:
# removes commas and converts value type to number
def clean_and_convert(columns):
    for col in columns:
        veh_hhld[col] = veh_hhld[col].astype(str).str.replace(',', '')
        veh_hhld[col] = pd.to_numeric(veh_hhld[col], errors='coerce')

In [458]:
clean_and_convert(low_inc_0)
clean_and_convert(high_inc_0)
clean_and_convert(low_inc_1p)
clean_and_convert(high_inc_1p)

In [459]:
veh_hhld['LOW_INC_0'] = veh_hhld[low_inc_0].sum(axis=1)
veh_hhld['HI_INC_0'] = veh_hhld[high_inc_0].sum(axis=1)
veh_hhld['LOW_INC_1P'] = veh_hhld[low_inc_1p].sum(axis=1)
veh_hhld['HI_INC_1P'] = veh_hhld[high_inc_1p].sum(axis=1)

In [460]:
veh_hhld = veh_hhld[['LOW_INC_0', 'HI_INC_0', 'LOW_INC_1P', 'HI_INC_1P']]

In [461]:
veh_hhld = veh_hhld.reset_index(drop=True)

In [462]:
## combines ACS data into one dataframe
acs = acs_dp05.merge(acs_s1501, on = 'CENSUS_TRACT', suffixes = ('_DP05', '_S1501'))
acs = acs.merge(acs_dp04, on = 'CENSUS_TRACT', suffixes = ('', '_DP04'))

In [463]:
acs = acs[['CENSUS_TRACT', 'MEDIAN_AGE', 'P_BACH_25P']]

In [464]:
acs = pd.concat([acs, veh_hhld], axis=1)

In [465]:
acs.head()

,CENSUS_TRACT,MEDIAN_AGE,P_BACH_25P,LOW_INC_0,HI_INC_0,LOW_INC_1P,HI_INC_1P
0,21067000101,31.0,32.675045,520.0,0.0,860.0,80.0
1,21067000102,52.4,53.358925,290.0,10.0,400.0,40.0
2,21067000200,24.1,42.355890,430.0,0.0,470.0,50.0
3,21067000300,36.8,45.487365,410.0,0.0,520.0,120.0
4,21067000400,32.8,30.097087,180.0,0.0,434.0,65.0


In [466]:
acs.to_csv('E:/CR2/Repos/TNC-Demand-Model-Southeast/Cleaned_Inputs/Intermediate/acs.csv')

In [467]:
# merges wac and acs dataframes into one

In [468]:
lehd = pd.read_csv("E:/CR2/Repos/TNC-Demand-Model-Southeast/Cleaned_Inputs/Intermediate/wac.csv")

In [469]:
lehd['trct_string'] = lehd['trct'].astype(str)

In [470]:
merged_data = acs.merge(lehd, left_on='CENSUS_TRACT', right_on='trct_string', how='left')

In [471]:
merged_data = merged_data[['CENSUS_TRACT', 'MEDIAN_AGE', 'LOW_INC_0', 'HI_INC_0', 'LOW_INC_1P', 'HI_INC_1P', 'P_BACH_25P', 'FOOD_EMP', 'RETAIL_EMP', 'OTHER_EMP', 'TOTAL_EMP_DEN']]

In [472]:
# placeholder for Logsum

In [473]:
merged_data['LOGSUM'] = 0

In [474]:
# placeholder for Airport

In [475]:
merged_data['AIR_F'] = 0

In [476]:
merged_data.loc[78, 'AIR_F'] = 1

In [477]:
# plapceholder for Tourist

In [478]:
merged_data['TOR_F'] = 0

In [479]:
# renamed column so we can merge this csv file with shapefile on QGIS on GEOID20 i.e. Census tract

In [480]:
merged_data = merged_data.rename(columns = {'CENSUS_TRACT' : 'GEOID20' })

In [481]:
merged_data.tail()

,GEOID20,MEDIAN_AGE,LOW_INC_0,HI_INC_0,LOW_INC_1P,HI_INC_1P,P_BACH_25P,FOOD_EMP,RETAIL_EMP,OTHER_EMP,TOTAL_EMP_DEN,LOGSUM,AIR_F,TOR_F
77,21067004205,42.9,0.0,0.0,360.0,180.0,47.007042,0,0,535,373.864430,0,0,0
78,21067004207,49.1,0.0,0.0,230.0,85.0,50.110497,193,65,1014,93.964689,0,1,0
79,21067004208,42.5,25.0,10.0,580.0,165.0,51.132139,1159,626,3059,1507.625272,0,0,0
80,21067004209,35.2,10.0,0.0,505.0,170.0,43.029016,79,28,83,293.663060,0,0,0
81,21067004210,33.0,19.0,0.0,480.0,170.0,53.177258,19,298,419,673.992674,0,0,0


In [482]:
# send merged ACS and WAC data to merged file

In [483]:
merged_data.to_csv('E:/CR2/Repos/TNC-Demand-Model-Southeast/Cleaned_Inputs/Intermediate/merged_fayette_data.csv')